<a href="https://colab.research.google.com/github/akshra09/Grammer-Scoring-Engine/blob/main/Grammer_scoring_Engine_Akshra_Verma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"akshraverma09","key":"bf042976131eaf510a8b28bef95e41b2"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download tapakah68/emotions-on-audio-dataset

Dataset URL: https://www.kaggle.com/datasets/tapakah68/emotions-on-audio-dataset
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)


In [4]:
!unzip emotions-on-audio-dataset.zip -d emotions_audio_data


Archive:  emotions-on-audio-dataset.zip
  inflating: emotions_audio_data/files/00026029e0--64991b6eef1fe70609d48edc/euphoric.wav  
  inflating: emotions_audio_data/files/00026029e0--64991b6eef1fe70609d48edc/joyfully.wav  
  inflating: emotions_audio_data/files/00026029e0--64991b6eef1fe70609d48edc/sad.wav  
  inflating: emotions_audio_data/files/00026029e0--64991b6eef1fe70609d48edc/surprised.wav  
  inflating: emotions_audio_data/files/00026029e0--64991b72e0daf97163c09c66/euphoric.wav  
  inflating: emotions_audio_data/files/00026029e0--64991b72e0daf97163c09c66/joyfully.wav  
  inflating: emotions_audio_data/files/00026029e0--64991b72e0daf97163c09c66/sad.wav  
  inflating: emotions_audio_data/files/00026029e0--64991b72e0daf97163c09c66/surprised.wav  
  inflating: emotions_audio_data/files/00026029e0--64991b7fd94c0d5726dec353/euphoric.wav  
  inflating: emotions_audio_data/files/00026029e0--64991b7fd94c0d5726dec353/joyfully.wav  
  inflating: emotions_audio_data/files/00026029e0--64991b7

In [1]:
!pip install -q openai-whisper
!pip install -q language-tool-python
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s

In [2]:

!pip install -q gradio


In [3]:

import whisper
import language_tool_python
import gradio as gr
import matplotlib.pyplot as plt
import tempfile
import os

# Fix matplotlib backend for Colab
import matplotlib
matplotlib.use('Agg')

# Score history
score_history = []

# Supported LanguageTool codes
lt_languages = {
    'en': 'en-US',
    'es': 'es',       # Spanish
    'fr': 'fr',       # French
    'de': 'de',       # German
    'it': 'it',       # Italian
    'pt': 'pt-PT',    # Portuguese
    # Add more if needed
}

In [4]:
class GrammarScorer:
    def __init__(self):
        print("Loading Whisper model...")
        self.model = whisper.load_model("base")
        print("Whisper model loaded.")

    def transcribe(self, audio_path):
        print(f"Transcribing: {audio_path}")
        result = self.model.transcribe(audio_path)
        text = result["text"]
        lang = result["language"]
        print(f"Detected language: {lang}, Text: {text}")
        return text, lang

    def grammar_score(self, text, lang_code):
        lt_code = lt_languages.get(lang_code, 'en-US')
        tool = language_tool_python.LanguageTool(lt_code)

        matches = tool.check(text)
        num_errors = len(matches)
        num_words = len(text.split())

        # Avoid division by 0
        error_rate = num_errors / max(num_words, 1)
        score = round(max(0, 10 - error_rate * 10), 2)

        feedback = [f"• {m.message} (\"{text[m.offset:m.offset + m.errorLength]}\")" for m in matches]

        print(f"Errors: {num_errors}, Words: {num_words}, Score: {score}")
        return score, feedback


    def process_audio(self, audio_path):
        text, lang = self.transcribe(audio_path)
        score, feedback = self.grammar_score(text, lang)
        score_history.append(score)
        return text, score, feedback, lang

In [5]:

def plot_score_chart():
    if not score_history:
        return None

    plt.figure(figsize=(5, 2))
    plt.plot(score_history, marker='o', color='blue')
    plt.title("Grammar Score History")
    plt.xlabel("Attempt")
    plt.ylabel("Score")
    plt.ylim(0, 10)
    plt.grid(True)

    tmpfile = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
    plt.savefig(tmpfile.name)
    plt.close()
    return tmpfile.name

In [6]:

scorer = GrammarScorer()

Loading Whisper model...


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 80.4MiB/s]


Whisper model loaded.


In [7]:
def analyze(audio):
    text, score, feedback, lang = scorer.process_audio(audio)
    chart_path = plot_score_chart()
    return text, score, "\n".join(feedback if feedback else ["No issues found."]), f"Detected: {lang.upper()}", chart_path

# Gradio UI


In [8]:

# Gradio UI
with gr.Blocks() as app:
    gr.Markdown("# 🗣️ Grammar Scoring Engine (Multilingual)")
    gr.Markdown("Upload or record your voice. We'll transcribe, detect grammar issues, and give a score + feedback!")

    with gr.Row():
        audio_input = gr.Audio(type="filepath", label="🎤 Record or Upload Audio")

    with gr.Row():
        transcript = gr.Textbox(label="📝 Transcription")
        score = gr.Number(label="✅ Grammar Score")

    feedback_box = gr.Textbox(label="📋 Feedback")
    lang_box = gr.Textbox(label="🌍 Detected Language")
    chart_output = gr.Image(label="📈 Score History")

    analyze_btn = gr.Button("Analyze")
    analyze_btn.click(fn=analyze, inputs=[audio_input], outputs=[transcript, score, feedback_box, lang_box, chart_output])

app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://79a3665f3419d70abc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
